In [7]:
import numpy as np
import pandas as pd
import operator
import pickle
import datetime
from datetime import timedelta
import math
from dateutil.relativedelta import relativedelta

In [2]:
## Reading MIMIC Data
data_dir = "../Data/"
drug_codes = pd.read_csv(data_dir+"DRGCODES.csv.gz", compression="gzip")
patients = pd.read_csv(data_dir+"PATIENTS.csv.gz", compression="gzip")

In [3]:
with open('../Data/Opioids/Prescriptions/prescriptions_with_narcotic_drugs.pickle', 'rb') as pickler:
    prescriptions_with_narcotic_drugs = pickle.load(pickler)
with open('../Data/Opioids/Prescriptions/prescriptions_with_anti_narcotic_drugs.pickle', 'rb') as pickler:
    prescriptions_with_anti_narcotic_drugs = pickle.load(pickler)
with open('../Data/Opioids/Prescriptions/prescriptions_with_mixed_drugs.pickle', 'rb') as pickler:
    prescriptions_with_mixed_drugs = pickle.load(pickler)
with open('../Data/Opioids/Prescriptions/prescriptions_with_both_drugs.pickle', 'rb') as pickler:
    prescriptions_with_both_drugs = pickle.load(pickler)

In [4]:
narcotic_drugs = ['oxymorphone', 'oxycodone', 'morphine', 'meperidine', 
                  'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine', 'buprenorphine', 'levorphanol', 'methadone']
anti_narcotic_drugs = ['methadone', 'naloxone', 'nalorphine']
both_drugs = narcotic_drugs + anti_narcotic_drugs

In [5]:
subjects_with_narcotic_drugs = set(prescriptions_with_narcotic_drugs["SUBJECT_ID"])
subjects_with_anti_narcotic_drugs = set(prescriptions_with_anti_narcotic_drugs["SUBJECT_ID"])
subjects_with_mixed_drugs = set(prescriptions_with_mixed_drugs["SUBJECT_ID"])
subjects_with_both_drugs = set(prescriptions_with_both_drugs["SUBJECT_ID"])

In [6]:
print("subjects_with_narcotic_drugs: %i" %(len(subjects_with_narcotic_drugs)))
print("subjects_with_anti_narcotic_drugs: %i" %(len(subjects_with_anti_narcotic_drugs)))
print("subjects_with_mixed_drugs (methadone): %i" %(len(subjects_with_mixed_drugs)))
print("subjects_with_both_drugs: %i" %(len(subjects_with_both_drugs)))

subjects_with_narcotic_drugs: 29959
subjects_with_anti_narcotic_drugs: 1437
subjects_with_mixed_drugs (methadone): 736
subjects_with_both_drugs: 29991


In [12]:
subjects_with_narcotic_and_anti_narcotic = subjects_with_narcotic_drugs.intersection(subjects_with_anti_narcotic_drugs)
print("subjects_with_narcotic_and_anti_narcotic: %i" %len(subjects_with_narcotic_and_anti_narcotic))

subjects_with_narcotic_and_anti_narcotic: 1405


In [13]:
patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [14]:
subjects_alive = patients[patients["EXPIRE_FLAG"] == 0]["SUBJECT_ID"]
subjects_expired = patients[patients["EXPIRE_FLAG"] == 1]["SUBJECT_ID"]
print ("Alive: %i; Expired: %i" %(len(set(subjects_alive)), len(set(subjects_expired))))

Alive: 30761; Expired: 15759


In [15]:
subjects_with_narcotic_drugs

{4,
 6,
 11,
 12,
 13,
 17,
 18,
 20,
 21,
 23,
 24,
 26,
 28,
 31,
 34,
 35,
 36,
 37,
 38,
 41,
 42,
 43,
 44,
 49,
 52,
 55,
 56,
 61,
 64,
 65,
 68,
 72,
 75,
 77,
 78,
 79,
 80,
 81,
 85,
 86,
 93,
 94,
 95,
 96,
 97,
 98,
 103,
 106,
 107,
 109,
 111,
 112,
 113,
 115,
 117,
 123,
 124,
 125,
 130,
 133,
 134,
 135,
 136,
 137,
 138,
 140,
 141,
 142,
 143,
 144,
 145,
 147,
 150,
 151,
 152,
 154,
 155,
 156,
 157,
 160,
 161,
 164,
 165,
 166,
 169,
 170,
 171,
 172,
 175,
 176,
 177,
 178,
 179,
 182,
 183,
 184,
 188,
 189,
 191,
 195,
 197,
 198,
 199,
 200,
 202,
 203,
 208,
 209,
 210,
 211,
 213,
 214,
 217,
 221,
 222,
 223,
 224,
 225,
 226,
 228,
 231,
 234,
 235,
 236,
 238,
 239,
 240,
 241,
 245,
 246,
 248,
 249,
 250,
 252,
 255,
 256,
 262,
 263,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 275,
 276,
 279,
 281,
 282,
 285,
 286,
 287,
 290,
 293,
 294,
 295,
 298,
 301,
 302,
 303,
 305,
 306,
 307,
 309,
 310,
 313,
 315,
 319,
 321,
 322,
 323,
 324,
 325,
 326

In [16]:
narcotic_subjects_live_count = {'alive':0, 'expired':0}
narcotic_subjects_live_count['alive'] = len(subjects_with_narcotic_drugs.intersection(subjects_alive))
narcotic_subjects_live_count['expired'] = len(subjects_with_narcotic_drugs.intersection(subjects_expired))
print("Narcotic Subjects:\n Alive: %i \t Expired: %i" 
      %(narcotic_subjects_live_count['alive'], narcotic_subjects_live_count['expired']))

Narcotic Subjects:
 Alive: 17888 	 Expired: 12071


In [17]:
anti_narcotic_subjects_live_count = {'alive':0, 'expired':0}
anti_narcotic_subjects_live_count['alive'] = len(subjects_with_anti_narcotic_drugs.intersection(subjects_alive))
anti_narcotic_subjects_live_count['expired'] = len(subjects_with_anti_narcotic_drugs.intersection(subjects_expired))
print("Anti Narcotic Subjects:\n Alive: %i \t Expired: %i" 
      %(anti_narcotic_subjects_live_count['alive'], anti_narcotic_subjects_live_count['expired']))

Anti Narcotic Subjects:
 Alive: 783 	 Expired: 654


In [18]:
narcotic_and_anti_narcotic_subjects_live_count = {'alive':0, 'expired':0}
narcotic_and_anti_narcotic_subjects_live_count['alive'] = len(subjects_with_narcotic_and_anti_narcotic.intersection(subjects_alive))
narcotic_and_anti_narcotic_subjects_live_count['expired'] = len(subjects_with_narcotic_and_anti_narcotic.intersection(subjects_expired))
print("Narcotic & Anti Narcotic Subjects:\n Alive: %i \t Expired: %i" 
      %(narcotic_and_anti_narcotic_subjects_live_count['alive'], narcotic_and_anti_narcotic_subjects_live_count['expired']))

Narcotic & Anti Narcotic Subjects:
 Alive: 757 	 Expired: 648


In [19]:
prescriptions_with_narcotic_drugs[prescriptions_with_narcotic_drugs["SUBJECT_ID"] == 11]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
614,2981121,11,194540,229441.0,2178-04-17 00:00:00,2178-05-07 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,406051201.0,5mg/325mg Tablet,1-2,TAB,1-2,TAB,PO
618,2981124,11,194540,NaN,2178-04-21 00:00:00,2178-04-22 00:00:00,MAIN,hydromorphone (dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),HYDR2I,004103,409131230.0,2mg/mL Syringe,1,mg,0.5,SYR,IV
622,2981129,11,194540,NaN,2178-04-21 00:00:00,2178-05-07 00:00:00,MAIN,hydromorphone (dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),HYDR2I,004103,409131230.0,2mg/mL Syringe,0.5-1,mg,0.25-0.5,SYR,IV
646,2981139,11,194540,NaN,2178-04-24 00:00:00,2178-04-24 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP4I,004072,409125830.0,4mg Syringe,1-5,mg,0.25-1.25,SYR,IV
648,2981140,11,194540,NaN,2178-04-24 00:00:00,2178-04-24 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,059794,409117830.0,50mg Syringe,12.5-25,mg,0.25-0.5,ml,IV
668,2981152,11,194540,NaN,2178-05-03 00:00:00,2178-05-11 00:00:00,MAIN,oxycodone (immediate release),OxycoDONE (Immediate Release),OxycoDONE (Immediate Release),OXYC5,004225,406055201.0,5mg Tablet,5,mg,1,TAB,PO


In [20]:
def calculate_days(row):
    start_date = row["STARTDATE"]
    start_date = start_date.split()[0].split("-", 4)
    #start_date = start_date.iloc[0]
    end_date = row["ENDDATE"]
    end_date = end_date.split()[0].split("-", 4)
    #end_date = end_date.iloc[0]
    no_of_days = datetime.date(int(end_date[0]), int(end_date[1]), int(end_date[2])) - datetime.date(int(start_date[0]), int(start_date[1]), int(start_date[2])) 
    return no_of_days.days

In [21]:
start_date_nan = prescriptions_with_narcotic_drugs[prescriptions_with_narcotic_drugs["STARTDATE"].isnull()]
end_date_nan = prescriptions_with_narcotic_drugs[prescriptions_with_narcotic_drugs["ENDDATE"].isnull()]
print(len(start_date_nan), len(end_date_nan))

36 271


In [22]:
narcotic_date_nan_subjects = set(start_date_nan["SUBJECT_ID"]).union(set(end_date_nan["SUBJECT_ID"]))
len(narcotic_date_nan_subjects)

188

In [23]:
narcotic_date_nan_subjects_alive = narcotic_date_nan_subjects.intersection(subjects_alive)
narcotic_date_nan_subjects_expired = narcotic_date_nan_subjects.intersection(subjects_expired)
print(len(narcotic_date_nan_subjects_alive), len(narcotic_date_nan_subjects_expired))

110 78


In [24]:
subject_13 = prescriptions_with_narcotic_drugs[prescriptions_with_narcotic_drugs["SUBJECT_ID"] == 13]
subject_13

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
42,2122405,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO
45,2122400,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV
46,2122399,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,004051,54354563.0,50 mg Syringe,12.5,mg,0.25,ml,IV
473,2123165,13,143045,263738.0,2167-01-12 00:00:00,2167-01-15 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO


In [26]:
narcotic_subject_df = pd.DataFrame(columns=prescriptions_with_narcotic_drugs.columns)
narcotic_subject_date_nan_df = pd.DataFrame(columns=prescriptions_with_narcotic_drugs.columns)
nan_count = 0
loop_count = 0
for index, row in prescriptions_with_narcotic_drugs.iterrows():
    loop_count += 1
    if (row["SUBJECT_ID"] not in narcotic_date_nan_subjects):
        row["NO_OF_DAYS_PRESCRIBED"] = calculate_days(row)
    else:
        nan_count += 1
        narcotic_subject_date_nan_df = narcotic_subject_date_nan_df.append(row)
    narcotic_subject_df = narcotic_subject_df.append(row)
    if (loop_count % 100 == 0):
        print("Iteration: %i" %(loop_count))
              
narcotic_subject_df.head()

Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900
Iteration: 1000
Iteration: 1100
Iteration: 1200
Iteration: 1300
Iteration: 1400
Iteration: 1500
Iteration: 1600
Iteration: 1700
Iteration: 1800
Iteration: 1900
Iteration: 2000
Iteration: 2100
Iteration: 2200
Iteration: 2300
Iteration: 2400
Iteration: 2500
Iteration: 2600
Iteration: 2700
Iteration: 2800
Iteration: 2900
Iteration: 3000
Iteration: 3100
Iteration: 3200
Iteration: 3300
Iteration: 3400
Iteration: 3500
Iteration: 3600
Iteration: 3700
Iteration: 3800
Iteration: 3900
Iteration: 4000
Iteration: 4100
Iteration: 4200
Iteration: 4300
Iteration: 4400
Iteration: 4500
Iteration: 4600
Iteration: 4700
Iteration: 4800
Iteration: 4900
Iteration: 5000
Iteration: 5100
Iteration: 5200
Iteration: 5300
Iteration: 5400
Iteration: 5500
Iteration: 5600
Iteration: 5700
Iteration: 5800
Iteration: 5900
Iteration: 6000
Iteration: 6100
Iteration: 6200
Iteration: 6300
I

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,NO_OF_DAYS_PRESCRIBED
42,2122405.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO,3.0
45,2122400.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV,3.0
46,2122399.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,004051,54354563.0,50 mg Syringe,12.5,mg,0.25,ml,IV,3.0
127,1213835.0,4.0,185777.0,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,guaifenesin-codeine phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,GGAC5L,045667,31867412.0,5ML UDCUP,5-10,ml,1-2,UDCUP,PO,7.0
173,2214000.0,6.0,107064.0,NaN,2175-05-30 00:00:00,2175-06-01 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine PCA,MORP50PCA,004067,338268975.0,50mg/50mL Syringe,1,mg,0.02,SYR,IVPCA,2.0


In [51]:
with open('../Data/Opioids/Subjects/narcotic_subject_df.pickle', 'wb') as pickler:
    pickle.dump(narcotic_subject_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
start_date_nan = prescriptions_with_anti_narcotic_drugs[prescriptions_with_anti_narcotic_drugs["STARTDATE"].isnull()]
end_date_nan = prescriptions_with_anti_narcotic_drugs[prescriptions_with_anti_narcotic_drugs["ENDDATE"].isnull()]
print(len(start_date_nan), len(end_date_nan))
anti_narcotic_date_nan_subjects = set(start_date_nan["SUBJECT_ID"]).union(set(end_date_nan["SUBJECT_ID"]))
print(len(anti_narcotic_date_nan_subjects))
anti_narcotic_date_nan_subjects_alive = anti_narcotic_date_nan_subjects.intersection(subjects_alive)
anti_narcotic_date_nan_subjects_expired = anti_narcotic_date_nan_subjects.intersection(subjects_expired)
print(len(anti_narcotic_date_nan_subjects_alive), len(anti_narcotic_date_nan_subjects_expired))

0 1
1
0 1


In [55]:
anti_narcotic_subject_df = pd.DataFrame(columns=prescriptions_with_narcotic_drugs.columns)
anti_narcotic_subject_date_nan_df = pd.DataFrame(columns=prescriptions_with_narcotic_drugs.columns)
nan_count = 0
loop_count = 0
for index, row in prescriptions_with_anti_narcotic_drugs.iterrows():
    loop_count += 1
    if (row["SUBJECT_ID"] not in anti_narcotic_date_nan_subjects):
        row["NO_OF_DAYS_PRESCRIBED"] = calculate_days(row)
    else:
        nan_count += 1
        anti_narcotic_subject_date_nan_df = anti_narcotic_subject_date_nan_df.append(row)
    anti_narcotic_subject_df = anti_narcotic_subject_df.append(row)
    if (loop_count % 100 == 0):
        print("Iteration: %i" %(loop_count))

with open('../Data/Opioids/Subjects/anti_narcotic_subject_df.pickle', 'wb') as pickler:
    pickle.dump(anti_narcotic_subject_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)
anti_narcotic_subject_df.head()

Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900
Iteration: 1000
Iteration: 1100
Iteration: 1200
Iteration: 1300
Iteration: 1400
Iteration: 1500
Iteration: 1600
Iteration: 1700
Iteration: 1800
Iteration: 1900
Iteration: 2000
Iteration: 2100
Iteration: 2200
Iteration: 2300
Iteration: 2400
Iteration: 2500
Iteration: 2600
Iteration: 2700
Iteration: 2800
Iteration: 2900
Iteration: 3000
Iteration: 3100
Iteration: 3200
Iteration: 3300
Iteration: 3400
Iteration: 3500
Iteration: 3600
Iteration: 3700
Iteration: 3800
Iteration: 3900
Iteration: 4000
Iteration: 4100
Iteration: 4200
Iteration: 4300
Iteration: 4400
Iteration: 4500
Iteration: 4600
Iteration: 4700
Iteration: 4800
Iteration: 4900
Iteration: 5000
Iteration: 5100
Iteration: 5200
Iteration: 5300
Iteration: 5400
Iteration: 5500
Iteration: 5600


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,NO_OF_DAYS_PRESCRIBED
1575,1386582.0,36.0,165660.0,241249.0,2134-05-15 00:00:00,2134-05-16 00:00:00,MAIN,naloxone,Naloxone,Naloxone,NALO4I,004510,409121201.0,0.4mg/1mL Vial,1,mg,2.5,mL,IV,1.0
3709,999273.0,78.0,100536.0,233150.0,2177-02-14 00:00:00,2177-02-18 00:00:00,MAIN,naloxone hcl,NaN,NaN,NALO4I,004510,74121201.0,0.4MG/1ML VL,1,mg,2.5,ml,IV DRIP,4.0
3710,999272.0,78.0,100536.0,233150.0,2177-02-14 00:00:00,2177-02-18 00:00:00,MAIN,naloxone hcl,NaN,NaN,NALO4I,004510,74121201.0,0.4MG/1ML VL,1,mg,2.5,ml,IV DRIP,4.0
3714,999256.0,78.0,100536.0,233150.0,2177-02-15 00:00:00,2177-02-15 00:00:00,MAIN,methadone hcl,Methadone HCl,Methadone HCl,METD5,004242,54855324.0,5MG TAB,5,mg,1,TAB,PO,0.0
3718,998562.0,78.0,100536.0,233150.0,2177-02-15 00:00:00,2177-02-18 00:00:00,MAIN,methadone hcl,Methadone HCl,Methadone HCl,METD5,004242,54855324.0,5MG TAB,50,mg,5,TAB,PO,3.0


In [57]:
anti_narcotic_subject_df["NO_OF_DAYS_PRESCRIBED"] += 1
anti_narcotic_subject_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,NO_OF_DAYS_PRESCRIBED
1575,1386582.0,36.0,165660.0,241249.0,2134-05-15 00:00:00,2134-05-16 00:00:00,MAIN,naloxone,Naloxone,Naloxone,NALO4I,004510,409121201.0,0.4mg/1mL Vial,1,mg,2.5,mL,IV,2.0
3709,999273.0,78.0,100536.0,233150.0,2177-02-14 00:00:00,2177-02-18 00:00:00,MAIN,naloxone hcl,NaN,NaN,NALO4I,004510,74121201.0,0.4MG/1ML VL,1,mg,2.5,ml,IV DRIP,5.0
3710,999272.0,78.0,100536.0,233150.0,2177-02-14 00:00:00,2177-02-18 00:00:00,MAIN,naloxone hcl,NaN,NaN,NALO4I,004510,74121201.0,0.4MG/1ML VL,1,mg,2.5,ml,IV DRIP,5.0
3714,999256.0,78.0,100536.0,233150.0,2177-02-15 00:00:00,2177-02-15 00:00:00,MAIN,methadone hcl,Methadone HCl,Methadone HCl,METD5,004242,54855324.0,5MG TAB,5,mg,1,TAB,PO,1.0
3718,998562.0,78.0,100536.0,233150.0,2177-02-15 00:00:00,2177-02-18 00:00:00,MAIN,methadone hcl,Methadone HCl,Methadone HCl,METD5,004242,54855324.0,5MG TAB,50,mg,5,TAB,PO,4.0


In [58]:
narcotic_subject_df["NO_OF_DAYS_PRESCRIBED"] += 1
narcotic_subject_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,NO_OF_DAYS_PRESCRIBED
42,2122405.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO,4.0
45,2122400.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV,4.0
46,2122399.0,13.0,143045.0,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,004051,54354563.0,50 mg Syringe,12.5,mg,0.25,ml,IV,4.0
127,1213835.0,4.0,185777.0,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,guaifenesin-codeine phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,GGAC5L,045667,31867412.0,5ML UDCUP,5-10,ml,1-2,UDCUP,PO,8.0
173,2214000.0,6.0,107064.0,NaN,2175-05-30 00:00:00,2175-06-01 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine PCA,MORP50PCA,004067,338268975.0,50mg/50mL Syringe,1,mg,0.02,SYR,IVPCA,3.0


In [59]:
with open('../Data/Opioids/Subjects/narcotic_subject_df.pickle', 'wb') as pickler:
    pickle.dump(narcotic_subject_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../Data/Opioids/Subjects/anti_narcotic_subject_df.pickle', 'wb') as pickler:
    pickle.dump(anti_narcotic_subject_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [92]:
no_of_days = datetime.date(2191, 3, 23) - datetime.date(2191, 3, 16)
print(end_date.get(0))

None


In [45]:
prescriptions_with_narcotic_drugs_grouped = prescriptions_with_narcotic_drugs.groupby('SUBJECT_ID').apply(
    lambda drug: drug[["DRUG"]].apply(lambda y: y.str.contains("|".join(narcotic_drugs)).sum()))
prescriptions_with_narcotic_drugs_grouped.head()

,DRUG
SUBJECT_ID,
4,1
6,4
11,6
12,3
13,4


In [32]:
total_subjects = set(prescriptions_with_both_drugs["SUBJECT_ID"])
print(len(total_subjects))
total_subjects_df = patients[patients["SUBJECT_ID"].isin(total_subjects)]
print("Opioid Subjects: %i" %len(total_subjects_df))
subjects_with_age = total_subjects_df[total_subjects_df.DOB.notnull and total_subjects_df.DOD.notnull()]
print("Subjects with age: %i" %len(subjects_with_age))

29991
Opioid Subjects: 29991
Subjects with age: 12077


In [34]:
subjects_with_no_age_set = total_subjects.symmetric_difference(set(subjects_with_age["SUBJECT_ID"]))
print("Subjects with no age: %i" %len(subjects_with_no_age_set))
subjects_with_no_age = patients[patients["SUBJECT_ID"].isin(subjects_with_no_age_set)]
print("Subjects with no age: %i" %len(subjects_with_no_age))

Subjects with no age: 17914
Subjects with no age: 17914


In [59]:
def calculate_age(row):
    start_date = row["DOB"]
    start_date = start_date.split()[0].split("-", 4)
    #start_date = start_date.iloc[0]
    #end_date = row["DOD"]
    if (pd.notnull(row["ENDDATE"])):
        end_date = row["ENDDATE"]
    elif (pd.notnull(row["STARTDATE"])):
        end_date = row["STARTDATE"]
    else:
        return 
    end_date = end_date.split()[0].split("-", 4)
    #end_date = end_date.iloc[0]
    age = relativedelta(datetime.date(int(end_date[0]), int(end_date[1]), int(end_date[2])), datetime.date(int(start_date[0]), int(start_date[1]), int(start_date[2]))) 
    return age.years

In [35]:
subjects_with_age["AGE"] = subjects_with_age.apply(calculate_age, axis=1)
print(len(subjects_with_age))
subjects_with_age.head()

12077


/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,23
12,246,263,M,2104-06-18 00:00:00,2168-06-13 00:00:00,2168-06-13 00:00:00,NaN,1,63
17,251,268,F,2132-02-21 00:00:00,2198-02-18 00:00:00,2198-02-18 00:00:00,NaN,1,65
22,628,665,M,2052-05-20 00:00:00,2120-02-04 00:00:00,2120-02-04 00:00:00,NaN,1,67
25,631,668,F,2096-08-18 00:00:00,2183-07-10 00:00:00,2183-07-10 00:00:00,2183-07-10 00:00:00,1,86


In [36]:
with open('../Data/Opioids/Subjects/subjects_with_age.pickle', 'wb') as pickler:
    pickle.dump(subjects_with_age, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
opioid_prescriptions_no_age = prescriptions_with_both_drugs[prescriptions_with_both_drugs["SUBJECT_ID"].isin(subjects_with_no_age_set)]
opioid_prescriptions_no_age

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
42,2122405,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,5.446502e+07,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO
45,2122400,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,7.417620e+07,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV
46,2122399,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,004051,5.435456e+07,50 mg Syringe,12.5,mg,0.25,ml,IV
127,1213835,4,185777,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,guaifenesin-codeine phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,GGAC5L,045667,3.186741e+07,5ML UDCUP,5-10,ml,1-2,UDCUP,PO
173,2214000,6,107064,NaN,2175-05-30 00:00:00,2175-06-01 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine PCA,MORP50PCA,004067,3.382690e+08,50mg/50mL Syringe,1,mg,0.02,SYR,IVPCA
177,2214004,6,107064,NaN,2175-05-30 00:00:00,2175-06-03 00:00:00,MAIN,hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,7.413123e+07,2mg/mL Syringe,0.5-1,mg,0.25-0.5,SYR,SC
205,2214010,6,107064,228232.0,2175-06-01 00:00:00,2175-06-15 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,4.060513e+08,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO
261,2214763,6,107064,NaN,2175-06-07 00:00:00,2175-06-14 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate (2-4mg),MORP2-4,004070,7.417623e+07,2-4mg Range,2-4,mg,0.2-0.4,SYR,IV
473,2123165,13,143045,263738.0,2167-01-12 00:00:00,2167-01-15 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,5.446502e+07,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO
490,709684,17,194023,NaN,2134-12-27 00:00:00,2134-12-29 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate (0.5-4mg),MORP0.5-4,004073,7.412590e+07,0.5-4mg Range,0.5-4,mg,0.05-0.4,SYR,IV


In [45]:
patients_dob = patients[patients["SUBJECT_ID"].isin(total_subjects)]
print(len(patients_dob))
patients_dob = patients_dob[['SUBJECT_ID', 'DOB']]
patients_dob.head()

29991


,SUBJECT_ID,DOB
0,249,2075-03-13 00:00:00
1,250,2164-12-27 00:00:00
3,252,2078-03-06 00:00:00
5,255,2109-08-05 00:00:00
6,256,2086-07-31 00:00:00


In [46]:
opioid_prescriptions_no_age = pd.merge(opioid_prescriptions_no_age, patients_dob, on="SUBJECT_ID", how="left")
print(len(opioid_prescriptions_no_age))
opioid_prescriptions_no_age.head()

124318


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,DOB
0,2122405,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,PERC,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO,2127-02-27 00:00:00
1,2122400,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV,2127-02-27 00:00:00
2,2122399,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,MEPE50I,004051,54354563.0,50 mg Syringe,12.5,mg,0.25,ml,IV,2127-02-27 00:00:00
3,1213835,4,185777,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,guaifenesin-codeine phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,GGAC5L,045667,31867412.0,5ML UDCUP,5-10,ml,1-2,UDCUP,PO,2143-05-12 00:00:00
4,2214000,6,107064,NaN,2175-05-30 00:00:00,2175-06-01 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine PCA,MORP50PCA,004067,338268975.0,50mg/50mL Syringe,1,mg,0.02,SYR,IVPCA,2109-06-21 00:00:00


In [60]:
opioid_prescriptions_no_age["AGE"] = opioid_prescriptions_no_age.apply(calculate_age, axis=1)
print(len(opioid_prescriptions_no_age))
opioid_prescriptions_no_age.head()

124318


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,...,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,DOB,AGE
0,2122405,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,...,004222,54465025.0,5mg/325mg Tab,1-2,TAB,1-2,TAB,PO,2127-02-27 00:00:00,39.0
1,2122400,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine Sulfate,...,004070,74176201.0,2mg Syringe,0.5-4.0,mg,0.25-2,SYR,IV,2127-02-27 00:00:00,39.0
2,2122399,13,143045,263738.0,2167-01-09 00:00:00,2167-01-12 00:00:00,MAIN,meperidine,Meperidine,Meperidine,...,004051,54354563.0,50 mg Syringe,12.5,mg,0.25,ml,IV,2127-02-27 00:00:00,39.0
3,1213835,4,185777,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,guaifenesin-codeine phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,...,045667,31867412.0,5ML UDCUP,5-10,ml,1-2,UDCUP,PO,2143-05-12 00:00:00,47.0
4,2214000,6,107064,NaN,2175-05-30 00:00:00,2175-06-01 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine PCA,...,004067,338268975.0,50mg/50mL Syringe,1,mg,0.02,SYR,IVPCA,2109-06-21 00:00:00,65.0


In [64]:
null_age_subjects = set(opioid_prescriptions_no_age[opioid_prescriptions_no_age["AGE"].isnull()]["SUBJECT_ID"])
print(len(null_age_subjects))
null_age_subjects

18


{6373,
 8587,
 12379,
 13867,
 16816,
 18244,
 19487,
 23054,
 26069,
 26361,
 28719,
 29449,
 32336,
 64997,
 72073,
 80914,
 92585,
 96544}

In [78]:
opioid_prescriptions_no_age[opioid_prescriptions_no_age["SUBJECT_ID"]==96544]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,...,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,DOB,AGE
119336,3937482,96544,188749,NaN,NaN,NaN,MAIN,morphine sulfate ir,Morphine Sulfate IR,Morphine Sulfate (Immediate Release),...,004091,5.402352e+07,15mg Tab,15-30,mg,1-2,TAB,PO/NG,2056-05-19 00:00:00,NaN
119397,3937980,96544,188749,NaN,2109-07-07 00:00:00,2109-07-13 00:00:00,MAIN,morphine sulfate ir,Morphine Sulfate IR,Morphine Sulfate (Immediate Release),...,004091,5.402352e+07,15mg Tab,15-30,mg,1-2,TAB,PO/NG,2056-05-19 00:00:00,53.0
119398,3937473,96544,188749,NaN,2109-07-12 00:00:00,2109-07-13 00:00:00,MAIN,hydromorphone (dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),...,004103,4.091312e+08,2mg/mL Syringe,0.5,mg,0.25,SYR,IV,2056-05-19 00:00:00,53.0
119399,3937476,96544,188749,NaN,2109-07-13 00:00:00,2109-07-13 00:00:00,MAIN,oxycodone-acetaminophen,Oxycodone-Acetaminophen,Oxycodone-Acetaminophen,...,004222,4.060513e+08,5mg/325mg Tablet,1-2,TAB,1-2,TAB,PO/NG,2056-05-19 00:00:00,53.0
119458,3937964,96544,188749,NaN,2109-07-04 00:00:00,2109-07-04 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine,...,004080,1.001902e+10,5 mg Vial,2-4,mg,0.4-0.8,VIAL,IV,2056-05-19 00:00:00,53.0
119459,3937968,96544,188749,NaN,2109-07-04 00:00:00,2109-07-07 00:00:00,MAIN,morphine sulfate,Morphine Sulfate,Morphine,...,004080,1.001902e+10,5 mg Vial,2-4,mg,0.4-0.8,VIAL,IV,2056-05-19 00:00:00,53.0


In [82]:
subjects_with_no_age = opioid_prescriptions_no_age.groupby('SUBJECT_ID').apply(lambda row: max(row["AGE"]))
subjects_with_no_age.to_csv("subjects_with_no_age.csv")
subjects_with_no_age = pd.read_csv("subjects_with_no_age.csv", header=None)
subjects_with_no_age.columns=["SUBJECT_ID", "AGE"]
subjects_with_no_age[subjects_with_no_age["SUBJECT_ID"]==96544]["AGE"] = 53
print(len(subjects_with_no_age))
subjects_with_no_age.head()

17914


/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,SUBJECT_ID,AGE
0,4,47.0
1,6,65.0
2,13,39.0
3,17,47.0
4,18,50.0


In [87]:
subjects_with_no_age = pd.read_csv("subjects_with_no_age.csv", header=None)
subjects_with_no_age.columns=["SUBJECT_ID", "AGE"]
subjects_with_no_age.head()

,SUBJECT_ID,AGE
0,4,47.0
1,6,65.0
2,13,39.0
3,17,47.0
4,18,50.0


In [88]:
temp_df = subjects_with_age[["SUBJECT_ID", "AGE"]]
print(len(temp_df))
temp_df.head()

12077


,SUBJECT_ID,AGE
1,250,23
12,263,63
17,268,65
22,665,67
25,668,86


In [91]:
subjects_age_df = temp_df.append(subjects_with_no_age)
print(len(subjects_age_df))

29991


In [94]:
subjects_age_df.head()

,SUBJECT_ID,AGE
1,250,23.0
12,263,63.0
17,268,65.0
22,665,67.0
25,668,86.0


In [95]:
with open('../Data/Opioids/Subjects/subjects_age_df.pickle', 'wb') as pickler:
    pickle.dump(subjects_age_df, pickler, protocol=pickle.HIGHEST_PROTOCOL)